# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
url ='https://www.imdb.com/list/ls025598828/' #Accedemos con una url distinta por que nos esta devolvindo un requests 403 la web
res = requests.get(url)
print(res.status_code)

200


In [3]:
soup = BeautifulSoup(res.content, 'html.parser')

In [ ]:
print(soup.prettify())

In [4]:
movies_nr = []
titles = []
years = []
ratings = []
runtimes = []
genres = []
descriptions = []
directors = []
stars = []
gross_values = []
votes_values = []

In [5]:
title = soup.find_all('h3',{'class':"lister-item-header"})

In [6]:
caja = soup.find_all('h3',{'class':"lister-item-header"})
caja[0]

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt0903747/">Breaking Bad</a>
<span class="lister-item-year text-muted unbold">(2008–2013)</span>
</h3>

In [7]:
len(caja)

100

In [8]:
title = caja[0].find_all('a')[0].text

In [9]:
titles = []
for i in range(len(caja)):
    title = caja[i].find_all('a')[0].text
    titles.append(title)
    


In [10]:
movie_nr = soup.find_all('span',{'class':"lister-item-index unbold text-primary"})


In [11]:

movies_nr = []
for i in range(len(movie_nr)):
    number = soup.find_all('span',{'class':"lister-item-index unbold text-primary"})[i].text
    movies_nr.append(number)
    


In [12]:
year = soup.find_all('span',{'class':"lister-item-year text-muted unbold"})


In [13]:
years = []
for i in range(len(year)):
    date = soup.find_all('span',{'class':"lister-item-year text-muted unbold"})[i].text
    years.append(date)
    


In [14]:
years = [i.replace('(', '') for i in years]
years = [i.replace(')', '') for i in years]
years = [i.replace('I', '') for i in years]
years = [i.replace(' ', '') for i in years]
years = [i.replace('X', '') for i in years]


In [16]:
rating = soup.find_all('span',{'class':"ipl-rating-star__rating"})


In [ ]:
ratings = []
while "Rate" in ratings:
    ratings.remove("Rate")
for item in ratings:
    print(item)

In [17]:
runtime = soup.find_all('span',{'class':"runtime"})

In [18]:
runtimes = []
for i in range(len(runtime)):
    duration = soup.find_all('span',{'class':"runtime"})[i].text
    runtimes.append(duration)
    

In [20]:
genre = soup.find_all('span',{'class':"genre"})

In [21]:
genres = []
for i in range(len(genre)):
    gen = soup.find_all('span',{'class':"genre"})[i].text
    genres.append(gen)
    

In [22]:
genres = [i.replace('\n', '') for i in genres]
genres = [i.strip() for i in genres]

In [23]:
description = soup.find_all('p',{'class':""})

In [24]:
description = soup.find_all('p',{'class':""})[1].text

In [25]:
# Create movie_data
movie_data = soup.findAll('div', attrs={'class': 'lister-item mode-detail'})

# Iterate through movie_data to append descriptions
for movie in movie_data:
    
    description = movie.find_all('p', class_ = '' )[0].text.replace('\n','')
    descriptions.append(description)
    

In [26]:
caja3 = soup.find_all('p')[2].text


In [29]:
# Create a list gathering movie directors
# Encontrar todos los elementos <a> con el atributo href que contienen /name
movie_directors = soup.find_all('a', href=True)
# Filtrar los elementos para obtener solo los enlaces de los directores
directors = [element.text.strip() for element in movie_directors if '/name' in element['href']]
    
# Imprimir los nombres de los directores
for director in directors:
    print(director)

Bryan Cranston
Aaron Paul
Anna Gunn
Betsy Brandt
Agnes Larsson
Pierre Coffin
Katie Crown
CW21
Luke Harrison
Mark Neveldine
Brian Taylor
Jason Statham
Amy Smart
Clifton Collins Jr.
Dwight Yoakam
Quentin Tarantino
Jamie Foxx
Christoph Waltz
Leonardo DiCaprio
Kerry Washington
Mark Neveldine
Brian Taylor
Jason Statham
Amy Smart
Carlos Sanz
Jose Pablo Cantillo
Chad Stahelski
Keanu Reeves
Michael Nyqvist
Alfie Allen
Willem Dafoe
Chad Stahelski
Keanu Reeves
Riccardo Scamarcio
Ian McShane
Ruby Rose
Todd Phillips
Jonah Hill
Miles Teller
Steve Lantz
Gregg Weiner
Quentin Tarantino
Samuel L. Jackson
Kurt Russell
Jennifer Jason Leigh
Walton Goggins
Srdjan Spasojevic
Srdjan 'Zika' Todorovic
Sergej Trifunovic
Jelena Gavrilovic
Slobodan Bestic
David Fincher
Morgan Freeman
Brad Pitt
Kevin Spacey
Andrew Kevin Walker
Quentin Tarantino
John Travolta
Uma Thurman
Samuel L. Jackson
Bruce Willis
Richard Phillips
Paul Greengrass
Tom Hanks
Barkhad Abdi
Barkhad Abdirahman
Catherine Keener
Brian De Palma
Al Pacin

In [30]:
star = soup.find_all('p',{'class':"text-muted text-small"})

In [31]:
# Create a list gathering movie directors
# Encontrar todos los elementos <a> con el atributo href que contienen /name
movie_stars = soup.find_all('a', href=True)
# Filtrar los elementos para obtener solo los enlaces de los directores
stars = [element.text.strip() for element in movie_stars if '/name' in element['href']]
    
# Imprimir los nombres de los directores
for star in stars:
    print(star)

Bryan Cranston
Aaron Paul
Anna Gunn
Betsy Brandt
Agnes Larsson
Pierre Coffin
Katie Crown
CW21
Luke Harrison
Mark Neveldine
Brian Taylor
Jason Statham
Amy Smart
Clifton Collins Jr.
Dwight Yoakam
Quentin Tarantino
Jamie Foxx
Christoph Waltz
Leonardo DiCaprio
Kerry Washington
Mark Neveldine
Brian Taylor
Jason Statham
Amy Smart
Carlos Sanz
Jose Pablo Cantillo
Chad Stahelski
Keanu Reeves
Michael Nyqvist
Alfie Allen
Willem Dafoe
Chad Stahelski
Keanu Reeves
Riccardo Scamarcio
Ian McShane
Ruby Rose
Todd Phillips
Jonah Hill
Miles Teller
Steve Lantz
Gregg Weiner
Quentin Tarantino
Samuel L. Jackson
Kurt Russell
Jennifer Jason Leigh
Walton Goggins
Srdjan Spasojevic
Srdjan 'Zika' Todorovic
Sergej Trifunovic
Jelena Gavrilovic
Slobodan Bestic
David Fincher
Morgan Freeman
Brad Pitt
Kevin Spacey
Andrew Kevin Walker
Quentin Tarantino
John Travolta
Uma Thurman
Samuel L. Jackson
Bruce Willis
Richard Phillips
Paul Greengrass
Tom Hanks
Barkhad Abdi
Barkhad Abdirahman
Catherine Keener
Brian De Palma
Al Pacin

In [32]:
caja2 = soup.find_all('p')[3].text

In [33]:
votes = soup.find_all('span',{'name':"nv"})

In [35]:
import re

# Lista para almacenar los valores con el símbolo "$"
gross_values = []

# Lista para almacenar los valores sin el símbolo "$"
votes_values = []

# Expresión regular para buscar el símbolo "$"
pattern = re.compile(r'\$')

# Iterar sobre los valores
for value in votes:
    text_value = value.get_text(strip=True)  # Obtener el texto del objeto Tag
    if pattern.search(text_value):  # Si la expresión regular encuentra el símbolo "$"
        gross_values.append(text_value)
    else:
        votes_values.append(text_value)



In [34]:
length = len(votes)
length

195

In [36]:
# Reduce movie_directors list
modified_stars = movie_stars[:100]

In [37]:
length = len(modified_stars)

In [38]:
# Reduce votes list
votes = votes_values[:100]

In [39]:
length = len(votes)

In [40]:
# Reduce movie_stars list
modified_directors = director[:100]

In [41]:
# Fill lists
# Valor de relleno
valor_relleno = 0

# Rellenar la lista hasta que tenga 100 valores
while len(ratings) < 100:
    ratings.append(valor_relleno)

In [42]:
valor_relleno = 0

# Rellenar la lista hasta que tenga 100 valores
while len(gross_values) < 100:
    gross_values.append(valor_relleno)

In [43]:
# Fill lists
# Valor de relleno
valor_relleno = 0

# Rellenar la lista hasta que tenga 100 valores
while len(runtimes) < 100:
    runtimes.append(valor_relleno)

In [44]:
diccionario_Imbd = {}
diccionario_Imbd['movie_nr'] = movies_nr
diccionario_Imbd['title'] = titles
diccionario_Imbd['year'] = years
diccionario_Imbd['rating'] = ratings
diccionario_Imbd['runtime'] = runtimes
diccionario_Imbd['genre'] = genres
diccionario_Imbd['description'] = descriptions
diccionario_Imbd['director'] = modified_directors
diccionario_Imbd['stars'] = star
diccionario_Imbd['votes_values'] = votes
diccionario_Imbd['gross_values'] = gross_values

In [45]:
Imbd_peliculas_df = pd.DataFrame(diccionario_Imbd)
Imbd_peliculas_df.head()

,movie_nr,title,year,rating,runtime,genre,description,director,stars,votes_values,gross_values
0,1.,Breaking Bad,2008–2013,0,45 min,"Crime, Drama, Thriller",A chemistry teacher diagnosed with inoperable ...,Danny Aiello,Danny Aiello,"2,134,787",$13.63M
1,2.,Minecraft,2009VideoGame,0,96 min,"Action, Adventure, Family",Welcome to the world of Minecraft where you ca...,Danny Aiello,Danny Aiello,"14,077",$162.81M
2,3.,Crank: Alto voltaje,2009,0,165 min,"Action, Crime, Thriller",Chelios faces a Chinese mobster who has stolen...,Danny Aiello,Danny Aiello,"158,925",$27.83M
3,4.,Django desencadenado,2012,0,88 min,"Drama, Western","With the help of a German bounty-hunter, a fre...",Danny Aiello,Danny Aiello,"1,694,318",$43.04M
4,5.,Crank: Veneno en la sangre,2006,0,101 min,"Action, Crime, Thriller",Professional assassin Chev Chelios learns his ...,Danny Aiello,Danny Aiello,"263,551",$92.03M


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here